In [13]:
### Data Wrangling 
import pandas as pd
import numpy as np
from collections import OrderedDict

### Modelling 
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

### Remove unnecessary warnings
import warnings
warnings.filterwarnings('ignore')

In [15]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)

In [21]:
# load data set
dataset_url = "https://raw.githubusercontent.com/rifqi-522/datamining/main/credit_score%20(1).csv"

In [22]:
df = pd.read_csv(dataset_url)
df.head()

,Unnamed: 0,kode_kontrak,pendapatan_setahun_juta,kpr_aktif,durasi_pinjaman_bulan,jumlah_tanggungan,rata_rata_overdue,risk_rating
0,1,AGR-000001,295,YA,48,5,61 - 90 days,4
1,2,AGR-000011,271,YA,36,5,61 - 90 days,4
2,3,AGR-000030,159,TIDAK,12,0,0 - 30 days,1
3,4,AGR-000043,210,YA,12,3,46 - 60 days,3
4,5,AGR-000049,165,TIDAK,36,0,31 - 45 days,2


In [23]:
#check number of rows and columns in dataset
df.shape

(900, 8)

In [24]:
#check number of rows and columns in dataset
df.shape

(900, 8)

In [25]:
# select the class feature
labels = df["risk_rating"]

In [26]:
labels

0      4
1      4
2      1
3      3
4      2
      ..
895    5
896    3
897    1
898    2
899    1
Name: risk_rating, Length: 900, dtype: int64

In [27]:
# create a dataframe with all training data except the target column
X = df.drop(columns=["risk_rating"])

# check that the target variable has been removed
X.head()

,Unnamed: 0,kode_kontrak,pendapatan_setahun_juta,kpr_aktif,durasi_pinjaman_bulan,jumlah_tanggungan,rata_rata_overdue
0,1,AGR-000001,295,YA,48,5,61 - 90 days
1,2,AGR-000011,271,YA,36,5,61 - 90 days
2,3,AGR-000030,159,TIDAK,12,0,0 - 30 days
3,4,AGR-000043,210,YA,12,3,46 - 60 days
4,5,AGR-000049,165,TIDAK,36,0,31 - 45 days


*    Transform feature *rata_rata_overdue* to numerical data type 
*    Split the numerical columns based on "range days"


```
# Range Days
['>90', '0-30', '31-45', '46-60', '61-90']
```

In [28]:
split_overdue_X = pd.get_dummies(X["rata_rata_overdue"], prefix="overdue")
X = X.join(split_overdue_X)

X = X.drop(columns = "rata_rata_overdue")


*    Then normalize "kpr_aktif" feature
*    Split the numerical columns based on "yes" or "no"


```
# KPR values
['yes', 'no']
```

In [29]:
# 
KPR_status = pd.get_dummies(X["kpr_aktif"], prefix="KPR")
X = X.join(KPR_status)

# remove "rata_rata_overdue" feature
X = X.drop(columns = "kpr_aktif")

In [30]:
X

,Unnamed: 0,kode_kontrak,pendapatan_setahun_juta,durasi_pinjaman_bulan,jumlah_tanggungan,overdue_0 - 30 days,overdue_31 - 45 days,overdue_46 - 60 days,overdue_61 - 90 days,overdue_> 90 days,KPR_TIDAK,KPR_YA
0,1,AGR-000001,295,48,5,0,0,0,1,0,0,1
1,2,AGR-000011,271,36,5,0,0,0,1,0,0,1
2,3,AGR-000030,159,12,0,1,0,0,0,0,1,0
3,4,AGR-000043,210,12,3,0,0,1,0,0,0,1
4,5,AGR-000049,165,36,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
895,896,AGR-010739,112,48,5,0,0,0,0,1,0,1
896,897,AGR-010744,120,48,2,0,0,1,0,0,0,1
897,898,AGR-010758,166,24,2,1,0,0,0,0,1,0
898,899,AGR-010775,196,48,0,0,1,0,0,0,1,0


*    Normalize feature
*    Scale these fatures with range value between 0 to 1

```
# features will be normalize
['pendapatan_setahun_juta', 'durasi_pinjaman_bulan', 'jumlah_tanggungan']
```



In [31]:
# normalize feature 'pendapatan_setahun_juta', 'durasi_pinjaman_bulan', 'jumlah_tanggungan'
old_normalize_feature_labels = ['pendapatan_setahun_juta', 'durasi_pinjaman_bulan', 'jumlah_tanggungan']
new_normalized_feature_labels = ['norm_pendapatan_setahun_juta', 'norm_durasi_pinjaman_bulan', 'norm_jumlah_tanggungan']
normalize_feature = df[old_normalize_feature_labels]

In [32]:
normalize_feature

,pendapatan_setahun_juta,durasi_pinjaman_bulan,jumlah_tanggungan
0,295,48,5
1,271,36,5
2,159,12,0
3,210,12,3
4,165,36,0
...,...,...,...
895,112,48,5
896,120,48,2
897,166,24,2
898,196,48,0


In [33]:
scaler = MinMaxScaler()

In [34]:
scaler.fit(normalize_feature)

MinMaxScaler()

In [35]:
normalized_feature = scaler.transform(normalize_feature)

In [36]:
normalized_feature_df = pd.DataFrame(normalized_feature, columns = new_normalized_feature_labels)

In [37]:
normalized_feature_df

,norm_pendapatan_setahun_juta,norm_durasi_pinjaman_bulan,norm_jumlah_tanggungan
0,0.978261,1.000000,0.833333
1,0.873913,0.666667,0.833333
2,0.386957,0.000000,0.000000
3,0.608696,0.000000,0.500000
4,0.413043,0.666667,0.000000
...,...,...,...
895,0.182609,1.000000,0.833333
896,0.217391,1.000000,0.333333
897,0.417391,0.333333,0.333333
898,0.547826,1.000000,0.000000


In [38]:
X = X.drop(columns = old_normalize_feature_labels)

In [39]:
X = X.join(normalized_feature_df)

In [40]:
X = X.join(labels)

In [41]:
X

,Unnamed: 0,kode_kontrak,overdue_0 - 30 days,overdue_31 - 45 days,overdue_46 - 60 days,overdue_61 - 90 days,overdue_> 90 days,KPR_TIDAK,KPR_YA,norm_pendapatan_setahun_juta,norm_durasi_pinjaman_bulan,norm_jumlah_tanggungan,risk_rating
0,1,AGR-000001,0,0,0,1,0,0,1,0.978261,1.000000,0.833333,4
1,2,AGR-000011,0,0,0,1,0,0,1,0.873913,0.666667,0.833333,4
2,3,AGR-000030,1,0,0,0,0,1,0,0.386957,0.000000,0.000000,1
3,4,AGR-000043,0,0,1,0,0,0,1,0.608696,0.000000,0.500000,3
4,5,AGR-000049,0,1,0,0,0,1,0,0.413043,0.666667,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,896,AGR-010739,0,0,0,0,1,0,1,0.182609,1.000000,0.833333,5
896,897,AGR-010744,0,0,1,0,0,0,1,0.217391,1.000000,0.333333,3
897,898,AGR-010758,1,0,0,0,0,1,0,0.417391,0.333333,0.333333,1
898,899,AGR-010775,0,1,0,0,0,1,0,0.547826,1.000000,0.000000,2
